<div class="alert alert-block alert-success">
<center><b>AIM404 : Contextual Bandits with Amazon SageMaker RL - Bandits model deployment with the end-to-end loop</b></center>
</div>

The above cells explained the individual steps in the training workflow. To train a model to convergence, we will continually train the model based on data collected with client application interactions. We demonstrate the continual training loop in a single cell below.

We include the evaluation step at each step before deployment to compare the model just trained (`last_trained_model_id`) against the model that is currently hosted (`last_hosted_model_id`). If you want the loops to finish faster, you can set `do_evaluation=False` in the cell below.

Details of each joining and training job can be tracked in `join_db` and `model_db` respectively. `model_db` also stores the evaluation scores. When you have multiple experiments, you can check their status in `experiment_db`.

In [ ]:
import yaml
import sys
import numpy as np
import time
import sagemaker
import pprint
import pandas as pd
sys.path.append('common')
sys.path.append('common/sagemaker_rl')
from misc import get_execution_role
from markdown_helper import *
from IPython.display import Markdown
from IPython.core.display import Image, display, HTML

In [ ]:
display(Image('images/AIM404-workflow.png'))

<div class="alert alert-block alert-danger"">
<b>IMPORTANT :</b> In order to speed the training process in the loop, we will run it in local mode true and soft deployment true. You need to edit the file <b>config-loop.yaml</b>
<ul>
                                            <li>local_mode: <b>true</b></li>
                                            <li>soft_deployment: <b>true</b></li>
</ul>
</div>

In [ ]:
sys.path.append('sim_app')
from statlog_sim_app import StatlogSimApp
from sim_app_utils import *
from orchestrator.workflow.manager.experiment_manager import ExperimentManager
with open('config-loop.yaml', 'r') as yaml_file:
    config = yaml.load(yaml_file)

do_evaluation = True

experiment_name = "AIM404-Loop" #YOUR EXPERIMENT NAME HERE 
bandits_experiment = ExperimentManager(config, experiment_id=experiment_name)

In [ ]:
start_time = time.time()
total_loops = 10 # Increase for higher accuracy
batch_size = 100 # for the warm start
rewards_list = []

local_mode = bandits_experiment.local_mode

# upload to s3
from sim_app_utils import *
warm_start_data_buffer = prepare_statlog_warm_start_data(data_file='sim_app/shuttle.trn', batch_size=batch_size)
bandits_experiment.ingest_joined_data(warm_start_data_buffer,ratio=0.8)

# first model training
bandits_experiment.initialize_first_model(input_data_s3_prefix=bandits_experiment.last_joined_job_train_data) 
# first model deployment
bandits_experiment.deploy_model(model_id=bandits_experiment.last_trained_model_id) 
# setup predictor for inference
predictor = bandits_experiment.predictor
sim_app = StatlogSimApp(predictor=predictor)

assert sim_app.num_actions == bandits_experiment.config["algor"]["algorithms_parameters"]["num_arms"]

batch_size = 500 # for the loops

for loop_no in range(total_loops):
    print(f"""
    #################
    #################
         Loop {loop_no+1}
    #################
    #################
    """)
    
    # Generate experiences and log them
    for i in range(batch_size):
        user_id, user_context = sim_app.choose_random_user()
        action, event_id, model_id, action_prob, sample_prob = predictor.get_action(obs=user_context.tolist())
        reward = sim_app.get_reward(user_id, action, event_id, model_id, action_prob, sample_prob, local_mode)
        rewards_list.append(reward)
    
    
    # publish rewards sum for this batch to CloudWatch for monitoring 
    bandits_experiment.cw_logger.publish_rewards_for_simulation(
        bandits_experiment.experiment_id,
        sum(rewards_list[-batch_size:])/batch_size
    )
    
    # Local/Athena join
    if local_mode:
        bandits_experiment.ingest_joined_data(sim_app.joined_data_buffer,ratio=0.85)
    else:
        print("Waiting for firehose to flush data to s3...")
        time.sleep(60) 
        rewards_s3_prefix = bandits_experiment.ingest_rewards(sim_app.rewards_buffer)
        bandits_experiment.join(rewards_s3_prefix, ratio=0.85)
    
    # Train 
    bandits_experiment.train_next_model(
        input_data_s3_prefix=bandits_experiment.last_joined_job_train_data)
    
    if do_evaluation:
    # Evaluate
        bandits_experiment.evaluate_model(
            input_data_s3_prefix=bandits_experiment.last_joined_job_eval_data,
            evaluate_model_id=bandits_experiment.last_trained_model_id)
        eval_score_last_trained_model = bandits_experiment.get_eval_score(
            evaluate_model_id=bandits_experiment.last_trained_model_id,
            eval_data_path=bandits_experiment.last_joined_job_eval_data)

        bandits_experiment.evaluate_model(
            input_data_s3_prefix=bandits_experiment.last_joined_job_eval_data,
            evaluate_model_id=bandits_experiment.last_hosted_model_id)

        eval_score_last_hosted_model = bandits_experiment.get_eval_score(
            evaluate_model_id=bandits_experiment.last_hosted_model_id, 
            eval_data_path=bandits_experiment.last_joined_job_eval_data)
    
        # Deploy
        if eval_score_last_trained_model <= eval_score_last_hosted_model:
            bandits_experiment.deploy_model(model_id=bandits_experiment.last_trained_model_id)
            print ('Eval score in this context is actually the cost. It is calculated as 1 - mean reward')
            print ('Meaning, we should deploy the new model only if its evaluation score is smaller, otherwise not')
            print ('Eval score of the new model:',eval_score_last_trained_model)
            print ('Eval score of the old model:',eval_score_last_hosted_model)
            print ('We deploy the model in loop {}'.format({loop_no+1}))
        else:
            print ('Eval score in this context is actually the cost. It is calculated as 1 - mean reward')
            print ('Meaning, we should deploy the new model only if its evaluation score is smaller, otherwise not')
            print ('Eval score of the new model:',eval_score_last_trained_model)
            print ('Eval score of the old model:',eval_score_last_hosted_model)
            print('Not deploying model in loop {}'.format({loop_no+1}))
    else:
        bandits_experiment.deploy_model(model_id=bandits_experiment.last_trained_model_id)
    
    sim_app.clear_buffer()

print(f"Total time taken to complete {total_loops} loops: {time.time() - start_time}")

<a id='visualization'></a>
## Visualization

You can visualize the model performance along the training loop by plotting the rolling mean reward across client interactions. Here rolling mean reward is calculated on the last `rolling_window` number of data instances, where each data instance corresponds to a single client interaction. 

<div class="alert alert-block alert-warning">

Note: The plot below cannot be generated if the notebook has been restarted after the execution of the cell above. 
</div/>

In [ ]:
%%time
import matplotlib.pyplot as plt
from pylab import rcParams
import pandas as pd
%matplotlib inline

def get_mean_reward(reward_lst, batch_size=batch_size):
    mean_rew=list()
    for r in range(len(reward_lst)):
        mean_rew.append(sum(reward_lst[:r+1]) * 1.0 / ((r+1)*batch_size))
    return mean_rew

rcParams['figure.figsize'] = 15, 10
lwd = 5
cmap = plt.get_cmap('tab20')
colors=plt.cm.tab20(np.linspace(0, 1, 20))

rolling_window = 100
rewards_df = pd.DataFrame(rewards_list, columns=['bandit']).rolling(rolling_window).mean()
rewards_df['oracle'] = sum(sim_app.opt_rewards) / len(sim_app.opt_rewards)

rewards_df.plot(y=['bandit','oracle'],linewidth=lwd)
plt.legend(loc=4, prop={'size': 20})
plt.tick_params(axis='both', which='major', labelsize=15)
plt.xlabel('Data instances (models were updated every %s data instances)' % batch_size, size=20)
plt.ylabel('Rolling Mean Reward', size=30)
plt.ylim(0,1.2)
plt.grid()
plt.show()

#### Get mean rewards

In [ ]:
rewards_df.bandit.mean()

If you didn't manage to finish the loop training, below is an example with 20 iterations of 500 for each

In [ ]:
display(Image('images/AIM404-reward-graph.png'))

<div class="alert alert-block alert-success">
    Here we visualize the reward after a loop of 20 iterations with 500 batches for each iteration</div>

<a id='clean-up'></a>
## Clean up

<div class="alert alert-block alert-warning">

If you want to start again the loop, you need to clean your experiment
</div/>


In [ ]:
bandits_experiment.clean_resource(experiment_id=bandits_experiment.experiment_id)

In [ ]:
bandits_experiment.clean_table_records(experiment_id=bandits_experiment.experiment_id)

<a id='clean-up'></a>
## What's next?

Now you can start to optimize the results above by tweaking configurations of your vowpal wabbit algorithms. Hints look for the **hyperparameters!**
* exploration_policy
* num_policies